In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
from datetime import datetime
from scipy.optimize import optimize
from scipy.stats import gaussian_kde
import torch.optim as optim
import torch.functional as F
from torch.utils.data import DataLoader, TensorDataset
from Code_analyse_OOP import DataCharger

En pratique, on dispose des classes de preprocessing déjà implémentées. On a donc un objet de type DataCharger à disposition, contenant comme argument différents types de données. On peut donc s'appuyer dessus pour garder les données dont on a besoin.
Ici, on a besoin d'un jeu de données contenant une seule agence, avec des données sur plusieurs années. Pour cela, on considère un objet de la classe DataCharger, ce qui revient à considérer l'argument self.data_years

Problème majeur : Pour estimer l'espérance par SAA, on a besoin de générer un grand nombre (de l'ordre d'au moins $10^3$) scénarios, sachant qu'un scénario correspond à l'ensemble des valeurs de flux net à la fin de chaque jour du mois. En prenant cette définition, les données historiques ne permettent donc d'obtenir que 3-4 scénarios par mois. L'idée est donc de concaténer l'ensemble des observations par mois, et ensuite d'estimer la distribution empirique par KDE pour ensuite générer des scénarios. Cependant, KDE risque de lisser les outliers, il faut donc certainement modéliser la distribution des outliers à part, et introduire une probabilité d'occurence au sein de chaque scénario.

Une autre possibilité consisterait à modéliser les flux comme série temporelle, à l'aide par exemple d'un processus ARMA, ARIMA... et d'échantillonner à partir de là ensuite (ce qui aurait l'avantage de conserver la structure temporelle des données).

In [6]:
class Simulation_Optimisation_lissage:
    def __init__(self, beta : float, lambda_1 : float, lambda_2 : float, gamma : float, C_t : float, h : float, alpha : float, agences : DataCharger) -> None:  # On commence par recenser les différents paramètres du modèle
        self.B = beta  # Paramètre de régularisation (de la softmax / softplus)
        self.lm1 = lambda_1  # Pénalité sur la rupture douce
        self.lm2 = lambda_2  # Pénalité sur la rupture dure
        self.C = C_t  # Coût de transport de l'agence
        self.h = h  # Taux d'intérêt journalier (lié au coût de refinancement/ coût d'opportunité)
        self.A = alpha  # Niveau de confiance sur le seuil smin
        self.object = agences  # On attribue l'objet de classe DataCharger à self.object
        self.data = None
        self.year = None
        self.mois_possibles = ["Janvier", "Février", "Mars", "Avril", "Mai", "Juin", "Juillet", "Août", "Septembre", "Octobre", "Novembre", "Décembre"]
        self.monthly_data = {x: {} for x in self.mois_possibles}  # Dictionnaire vide dans lequel on va stocker, par mois, les données associées
        self.kde_estimate = None

    def select_data(self):
        liste_agences, liste_annee = self.object.liste_annees_agences_data()
        agence = int(input("Choisissez une seule agence parmi la liste précédente: "))
        self.object.change_agence_year_choice(agence = agence)
        self.data = self.object.data_years.copy()
        self.data = self.data.sort_values("date_heure_operation")  # Pour s'assurer d'avoir des données dans le bon ordre temporel
        self.data["flux_net"] = self.data.groupby("jour")["montant_operation"].sum()
        self.year = self.data["date_heure_operation"].dt.year.unique().tolist()  # Renvoie une liste des années dispos
        print(f"La donnée pour l'agence {agence} sur l'ensemble des années disponibles a été chargée dans self.data")

    def softmax(self, arr : np.ndarray):  # Régularisation des indicatrices
        return 1 / (1+ np.exp(-self.B*arr))

    def softplus(self, arr : np.ndarray):  # Régularisation des rampes (x+)
        return 1/(self.B) * np.log(1 + np.exp(self.B*arr))

    def retrieve_month_data_by_year(self):  # On suppose disposer d'un dataset qui contient une seule agence sur plusieurs années
        for year in self.year:
            data_year = self.data[self.data["date_heure_operation"].dt.year == year]
            for i,mois in enumerate(self.mois_possibles):
                flux_mois_annee = np.array(data_year[data_year["date_heure_operation"].dt.month == i+1]["flux_net"])
                if flux_mois_annee :
                    self.monthly_data[mois][year] = flux_mois_annee
        return self.monthly_data 
        
    # La fonction précédente renvoie donc un dictionnaire par mois (Janvier, Février...), où chaque mois est lui-même associé à un dictionnaire
    # par année (par exemple, Janvier 2022, Janvier 2023...)
        
    def kde_month_estimate(self, bw_method = 'scott'):
        kde_estimate = {}  # Dictionnaire qui va stocker la loi empirique par mois
        for month, data_year in self.monthly_data.items():
            complete_month = []   # On crée une liste des observations historiques complètes pour chaque mois
            for year, data_array in data_year.items():
                complete_month.append(data_array)
            complete_month = np.array(complete_month)
            complete_month = complete_month.flatten()

            if len(complete_month) > 1:
                kde_month = gaussian_kde(complete_month, bw_method = bw_method)
                kde_estimate[month] = kde_month
            else:
                print(f"Manque de données pour une estimation non-paramétrique par KDE pour le mois {month}")
        print("Estimation empirique par kde des lois par mois terminée")
        self.kde_estimate = kde_estimate  # Renvoie un dictionnaire des lois empiriques

    def sample_from_estimate(self, month : int, n_samples: int = 21):  # Permet de générer des scénarios pour un mois
        month_trad = self.mois_possibles[month-1]   # n_samples représente le nombre de jours ouvrés dans le mois (proche de 21 par défaut)
        if month_trad not in self.kde_estimate:
            raise ValueError(f"Aucune estimation KDE disponible pour le moi de {month_trad}")
        else:
            return self.kde_estimate[month_trad].resample(n_samples).reshape(-1)  # A faire attention pour la dimension...

    def sample_outlier(self):
        # On va prendre comme outlier toute transaction qui dépasse 95% des quantiles
        quantile_95 = self.
        
    
            
        
        
            
        
        